In [60]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline

In [86]:
def write_answer(file_name, *answer):
    print(*answer, file=open(file_name, "w"))
    

### 1.
Загрузите датасет. smsspamcollection.zip

### 2.
Считайте датасет в Python (можете сразу грузить все в память, выборка небольшая), выясните, что используется в качестве разделителей и как проставляются метки классов.



In [12]:
data_name = "SMSSpamCollection.txt"
data = ""
with open(data_name, "r", encoding='utf-8') as file:
    all_data = file.read().splitlines()
all_data[0]

'ham\tGo until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...'

In [17]:
res = np.array(list(line.split('\t') for line in all_data))

In [47]:
data = pd.DataFrame(res)
data.columns = ['label', "text"]
data["label"] = (data.label == "spam").astype(int)
data.head()

,label,text
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."


### 3.
Подготовьте для дальнейшей работы два списка: список текстов в порядке их следования в датасете и список соответствующих им меток классов. В качестве метки класса используйте 1 для спама и 0 для "не спама".




In [48]:
label = data.label
text = data.text

In [49]:
data.label.value_counts()

0    4827
1     747
Name: label, dtype: int64

### 4.
Используя sklearn.feature_extraction.text.CountVectorizer со стандартными настройками, получите из списка текстов матрицу признаков X.




In [57]:
counter = CountVectorizer()
X_count = counter.fit_transform(text)
X_count

<5574x8713 sparse matrix of type '<class 'numpy.int64'>'
	with 74169 stored elements in Compressed Sparse Row format>

In [67]:
from sklearn.linear_model import LogisticRegression
import sklearn.model_selection
import warnings
warnings.filterwarnings('ignore')

### 5.
Оцените качество классификации текстов с помощью LogisticRegression() с параметрами по умолчанию, используя sklearn.cross_validation.cross_val_score и посчитав среднее арифметическое качества на отдельных fold'ах. Установите random_state=2. Параметр cv задайте равным 10. В качестве метрики качества используйте f1-меру. Получившееся качество - один из ответов, которые потребуются при сдаче задания. 

Ответ округлить до 1 знака после запятой.

In [111]:
pipeline = Pipeline(
    [("counter", CountVectorizer()),
     ("classifier", LogisticRegression())]
    )

In [112]:
np.random.seed(2)
result = sklearn.model_selection.cross_val_score(pipeline, text, label, scoring="f1",cv=10)

In [113]:
answ_1 = round(result.mean(), 1)
print(answ_1)
write_answer("1.txt", answ_1)

0.9


### 6.
А теперь обучите классификатор на всей выборке и спрогнозируйте с его помощью класс для следующих сообщений:

Прогнозы классификатора (0 - не спам, 1 - спам), записанные через пробел, будут ответом в одном из вопросов ниже.

In [88]:
X_test = [
    "FreeMsg: Txt: CALL to No: 86888 & claim your reward of 3 hours talk time to use from your phone now! Subscribe6GB",
    "FreeMsg: Txt: claim your reward of 3 hours talk time",
    "Have you visited the last lecture on physics?",
    "Have you visited the last lecture on physics? Just buy this book and you will have all materials! Only 99$",
    "Only 99$"
]

In [90]:
%%time
model = pipeline.fit(text, label)

Wall time: 157 ms


In [98]:
ans_2 = model.predict(X_test)
print(*ans_2)
write_answer("2.txt", *ans_2)

1 1 0 0 0


### 7.
Задайте в CountVectorizer параметр ngram_range=(2,2), затем ngram_range=(3,3), затем ngram_range=(1,3). Во всех трех случаях измерьте получившееся в кросс-валидации значение f1-меры, округлите до второго знака после точки, и выпишете результаты через пробел в том же порядке. 

В данном эксперименте мы пробовали добавлять в признаки n-граммы для разных диапазонов n - только биграммы, только триграммы, и, наконец, все вместе - униграммы, биграммы и триграммы. Обратите внимание, что статистики по биграммам и триграммам намного меньше, поэтому классификатор только на них работает хуже. В то же время это не ухудшает результат сколько-нибудь существенно, если добавлять их вместе с униграммами, т.к. за счет регуляризации линейный классификатор не склонен сильно переобучаться на этих признаках.




In [99]:
ngram_range=[(2,2), (3,3), (1,3)]

res_3 = []
for ngram in ngram_range:
    pipeline = Pipeline(
        [("counter", CountVectorizer(ngram_range=ngram)),
         ("classifier", LogisticRegression())]
    )
    result = sklearn.model_selection.cross_val_score(pipeline, text, label, scoring="f1",cv=10).mean()
    res_3.append(round(result, 2))
print(*res_3)

0.82 0.73 0.93


In [100]:
write_answer("3.txt", *res_3)

### 8.
Повторите аналогичный п.7 эксперимент, используя вместо логистической регрессии MultinomialNB(). Обратите внимание, насколько сильнее (по сравнению с линейным классификатором) наивный Байес страдает от нехватки статистики по биграммам и триграммам.

По какой-то причине обучение наивного байесовского классификатора через Pipeline происходит с ошибкой. Чтобы получить правильный ответ, отдельно посчитайте частоты слов и обучите классификатор.




In [101]:
from sklearn.naive_bayes import MultinomialNB

In [121]:
res_4 = []
for ngram in ngram_range:
    np.random.seed(2)
    pipeline = Pipeline(
        [("counter", CountVectorizer(ngram_range=ngram)),
         ("classifier", MultinomialNB())]
    )
    result = sklearn.model_selection.cross_val_score(pipeline, text, label, scoring="f1",cv=10).mean()
    res_4.append(round(result, 2))
print(*res_4)

0.93 0.87 0.95


In [120]:
res_4 = []
for ngram in ngram_range:
    np.random.seed(2)
    X_count = CountVectorizer(ngram_range=ngram).fit_transform(text)
    result = sklearn.model_selection.cross_val_score(
        MultinomialNB(),
        X_count,
        label,
        scoring="f1",
        cv=10
    ).mean()
    
    res_4.append(round(result, 2))
print(*res_4)

0.65 0.38 0.89


In [116]:
write_answer("4.txt", *res_4)

### 9.
Попробуйте использовать в логистической регрессии в качестве признаков $Tf*idf$ из TfidfVectorizer на униграммах. 

Повысилось или понизилось качество на кросс-валидации по сравнению с CountVectorizer на униграммах? (напишите в файле с ответом 1 если повысилось, -1 если понизилось, и 0 если изменилось не более чем на 0.01). 

Обратите внимание, что результат перехода к $tf*idf$ не всегда будет таким - если вы наблюдаете какое-то явление на одном датасете, не надо сразу же его обобщать на любые данные.

In [104]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [105]:
res_5 = []
for ngram in ngram_range:
    pipeline = Pipeline(
        [("counter", TfidfVectorizer(ngram_range=ngram)),
         ("classifier", LogisticRegression())]
    )
    result = sklearn.model_selection.cross_val_score(pipeline, text, label, scoring="f1",cv=10).mean()
    res_5.append(round(result, 2))
print(*res_5)

0.45 0.22 0.8


In [108]:
res_5 = np.array(res_5)
res_3 = np.array(res_3)
res_5 - res_3

array([-0.37, -0.51, -0.13])

In [109]:
write_answer("5.txt", -1)